# Setup

## Library imports

In [1]:
# !pip install imbalanced-learn
from imblearn.over_sampling import SMOTE
from sklearn import linear_model
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.preprocessing import MinMaxScaler
import statsmodels.api as sm 
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns 


## Read in data

### Define columns considered

In [2]:
REQUIRED_COLUMNS = [
    "ORIG_RATE",
    "ORIG_AMOUNT",
    "ORIG_TERM",
    "OCLTV",
    "NUM_BO",
    "DTI",
    "CSCORE_B",
    "CSCORE_C",
    "FTHB_FLAG",
    "PUR_Cash_out",
    "PUR_Refinance",
    "PUR_Purchase",
    "PRO_Condominium",
    "PRO_Co_operative",
    "PRO_Planned_Urban",
    "PRO_Manufact_Home",
    "PRO_Single_Family",
    "NUM_UNIT",
    "OCC_Principal",
    "OCC_Second",
    "OCC_Investor",
    "MI_PCT",
    "MI_Borrower",
    "MI_Lender",
    "MI_Investor",
    "DLQ_FLAG"
]

#### Read and process dataframe 

In [3]:
#### Read in processed dataset
complete_processed_dataset = pd.read_csv("../../../2023_stat.csv")
model_data = complete_processed_dataset[REQUIRED_COLUMNS]

#### Deal with datetime cols "ORIG_DTE","FRST_DTE"
model_data["ORIG_DTE_YR"] = pd.to_datetime(complete_processed_dataset["ORIG_DTE"]).dt.year - 2000
model_data["ORIG_DTE_MONTH"] = pd.to_datetime(complete_processed_dataset["ORIG_DTE"]).dt.month
model_data["FRST_DTE_YR"] = pd.to_datetime(complete_processed_dataset["FRST_DTE"]).dt.year - 2000
model_data["FRST_DTE_MONTH"] = pd.to_datetime(complete_processed_dataset["FRST_DTE"]).dt.month

#### Fill in missing data
print(model_data.isna().sum())
for col in ["DTI", "CSCORE_B", "CSCORE_C"]: #  this should be dealt in the pre-processing
    col_median=model_data[col].median()
    model_data[col] = model_data[col].fillna(col_median)
#### Check that no more NA values are remaining
print(sum(model_data.isna().sum()))

#### Set X, Y matrices
Y = model_data['DLQ_FLAG'].astype('float')
X = model_data.drop(columns=["MI_Investor","DLQ_FLAG"])

/var/folders/js/zv14t_p906xd0p0p9fq7clrw0000gn/T/ipykernel_731/1192086454.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  model_data["ORIG_DTE_YR"] = pd.to_datetime(complete_processed_dataset["ORIG_DTE"]).dt.year - 2000
/var/folders/js/zv14t_p906xd0p0p9fq7clrw0000gn/T/ipykernel_731/1192086454.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  model_data["ORIG_DTE_MONTH"] = pd.to_datetime(complete_processed_dataset["ORIG_DTE"]).dt.month
/var/folders/js/zv14t_p906xd0p0p9fq7clrw0000gn/T/ipykernel_731/1192

ORIG_RATE               0
ORIG_AMOUNT             0
ORIG_TERM               0
OCLTV                   0
NUM_BO                  0
DTI                    37
CSCORE_B             1552
CSCORE_C              971
FTHB_FLAG               0
PUR_Cash_out            0
PUR_Refinance           0
PUR_Purchase            0
PRO_Condominium         0
PRO_Co_operative        0
PRO_Planned_Urban       0
PRO_Manufact_Home       0
PRO_Single_Family       0
NUM_UNIT                0
OCC_Principal           0
OCC_Second              0
OCC_Investor            0
MI_PCT                  0
MI_Borrower             0
MI_Lender               0
MI_Investor             0
DLQ_FLAG                0
ORIG_DTE_YR             0
ORIG_DTE_MONTH          0
FRST_DTE_YR             0
FRST_DTE_MONTH          0
dtype: int64
0


# Train/Test Split

In [4]:
COLUMNS_TO_STANDARDIZE = [
    "ORIG_RATE",
    "ORIG_AMOUNT",
    "ORIG_TERM",
    "OCLTV",
    "NUM_BO",
    "DTI",
    "CSCORE_B",
    "CSCORE_C",
    "NUM_UNIT",
    "ORIG_DTE_YR",
    "ORIG_DTE_MONTH",
    "FRST_DTE_YR",
    "FRST_DTE_MONTH"
]
scaler = MinMaxScaler()
X = pd.DataFrame(scaler.fit_transform(X), columns=X.columns)
X


,ORIG_RATE,ORIG_AMOUNT,ORIG_TERM,OCLTV,NUM_BO,DTI,CSCORE_B,CSCORE_C,FTHB_FLAG,PUR_Cash_out,...,OCC_Principal,OCC_Second,OCC_Investor,MI_PCT,MI_Borrower,MI_Lender,ORIG_DTE_YR,ORIG_DTE_MONTH,FRST_DTE_YR,FRST_DTE_MONTH
0,0.618632,0.093135,1.0,0.563107,0.00,0.564516,0.891008,0.880240,0.0,0.0,...,0.0,0.0,1.0,0.000,0.0,0.0,1.000000,0.000000,0.5,0.181818
1,0.601164,0.072012,1.0,0.466019,0.00,0.645161,0.752044,0.727545,0.0,1.0,...,1.0,0.0,0.0,0.000,0.0,0.0,0.666667,1.000000,0.5,0.090909
2,0.781659,0.093135,1.0,0.708738,0.25,0.403226,0.896458,0.907186,0.0,0.0,...,0.0,0.0,1.0,0.000,0.0,0.0,0.666667,1.000000,0.5,0.090909
3,0.655022,0.167547,1.0,0.805825,0.00,0.758065,0.866485,0.853293,1.0,0.0,...,1.0,0.0,0.0,0.300,1.0,0.0,0.666667,1.000000,0.5,0.090909
4,0.691412,0.162266,1.0,0.466019,0.25,0.580645,0.405995,0.502994,0.0,0.0,...,1.0,0.0,0.0,0.000,0.0,0.0,0.666667,1.000000,0.5,0.090909
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
970181,0.344978,0.193471,1.0,0.757282,0.25,0.580645,0.754768,0.799401,1.0,0.0,...,1.0,0.0,0.0,0.000,0.0,0.0,1.000000,0.909091,1.0,0.000000
970182,0.708879,0.070571,1.0,0.640777,0.00,0.483871,0.910082,0.901198,1.0,0.0,...,1.0,0.0,0.0,0.000,0.0,0.0,1.000000,0.909091,1.0,0.000000
970183,0.764192,0.342775,1.0,0.582524,0.00,0.790323,0.891008,0.880240,0.0,0.0,...,1.0,0.0,0.0,0.000,0.0,0.0,1.000000,0.818182,0.5,1.000000
970184,0.746725,0.152664,1.0,0.922330,0.25,0.709677,0.678474,0.685629,1.0,0.0,...,1.0,0.0,0.0,0.875,1.0,0.0,1.000000,0.909091,1.0,0.000000


In [5]:
split = 0.7
X_train, Y_train = X[:int(len(X)*split)], Y[:int(len(X)*split)]
X_test, Y_test = X[int(len(X)*split):], Y[int(len(X)*split):]

# Model fitting

In [6]:
def fit_logistic_regression(
    train: list[pd.DataFrame, pd.DataFrame],
    test: list[pd.DataFrame, pd.DataFrame],
    model_config: dict = {}
):
    logistic_model = linear_model.LogisticRegression(
        C = model_config.get("C", 1),
        solver=model_config.get("solver", "liblinear"),
        random_state=model_config.get("random_state", 0),
        class_weight=model_config.get("class_weight", None)
    )
    logistic_model.fit(train[0], train[1])

    print("Training accuracy:", logistic_model.score(train[0], train[1]))
    print("Testing accuracy:", logistic_model.score(test[0], test[1]))
    print(
        classification_report(
            y_pred=logistic_model.predict(test[0]),
            y_true=test[1]
        )
    )

    return logistic_model
    

## Sanity checking

In [7]:
X_small, Y_small = X[:int(0.001*len(X))], Y[:int(0.001*len(Y))]
fit_logistic_regression(
    [X_small, Y_small],
    [X_small, Y_small],
    model_config={
        "class_weight": {
            0 :  len(Y_small)/(2*(len(Y_small)-sum(Y_small))),
            1 :  len(Y_small)/(2* sum(Y_small))
        }
    }
)

Training accuracy: 0.6731958762886598
Testing accuracy: 0.6731958762886598
              precision    recall  f1-score   support

         0.0       0.97      0.67      0.80       915
         1.0       0.11      0.65      0.19        55

    accuracy                           0.67       970
   macro avg       0.54      0.66      0.49       970
weighted avg       0.92      0.67      0.76       970



LogisticRegression(C=1,
                   class_weight={0: 0.5300546448087432, 1: 8.818181818181818},
                   random_state=0, solver='liblinear')

## Evaluate model

In [8]:
# Weighted classes
fit_logistic_regression(
    [X, Y],
    [X_test, Y_test],
    model_config={
        "class_weight": {
            0 :  len(Y_train)/(2*(len(Y_train)-sum(Y_train))),
            1 :  len(Y_train)/(2* sum(Y_train))
        }
    }
)

Training accuracy: 0.7386037316555795
Testing accuracy: 0.8699322467154087
              precision    recall  f1-score   support

         0.0       0.99      0.88      0.93    285599
         1.0       0.05      0.32      0.08      5457

    accuracy                           0.87    291056
   macro avg       0.52      0.60      0.51    291056
weighted avg       0.97      0.87      0.91    291056



LogisticRegression(C=1,
                   class_weight={0: 0.5179177438666331, 1: 14.452649499893594},
                   random_state=0, solver='liblinear')

In [9]:
## SMOTE
oversample = SMOTE()
X_train_smote, Y_train_smote = oversample.fit_resample(X_train, Y_train)
complete_model = fit_logistic_regression(
    [X_train_smote, Y_train_smote],
    [X_test, Y_test],
    # model_config={"C" : 100,}
)

Training accuracy: 0.6610728530356067
Testing accuracy: 0.8365263042163708
              precision    recall  f1-score   support

         0.0       0.99      0.85      0.91    285599
         1.0       0.04      0.37      0.08      5457

    accuracy                           0.84    291056
   macro avg       0.51      0.61      0.49    291056
weighted avg       0.97      0.84      0.89    291056



# Backward Elimination

In [10]:
model = sm.Logit(Y_train, X_train).fit()
model.summary()

Optimization terminated successfully.
         Current function value: 0.139955
         Iterations 8


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:               DLQ_FLAG   No. Observations:               679130
Model:                          Logit   Df Residuals:                   679103
Method:                           MLE   Df Model:                           26
Date:                Fri, 18 Oct 2024   Pseudo R-squ.:                 0.06927
Time:                        03:07:14   Log-Likelihood:                -95048.
converged:                       True   LL-Null:                   -1.0212e+05
Covariance Type:            nonrobust   LLR p-value:                     0.000
=====================================================================================
                        coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------
ORIG_RATE             0.5706      0.079      7.250      0.000       0.416       0.725
ORIG_AMOUNT           1.4430      0.086     16.854      0.000       1.275       1.611
ORIG_TERM             0.3645      0.053      6.865      0.000       0.260       0.469
OCLTV                 0.1393      0.060      2.307      0.021       0.021       0.258
NUM_BO               -1.3684      0.055    -25.042      0.000      -1.475      -1.261
DTI                   0.6887      0.050     13.694      0.000       0.590       0.787
CSCORE_B             -4.1210      0.094    -43.862      0.000      -4.305      -3.937
CSCORE_C             -1.1980      0.087    -13.810      0.000      -1.368      -1.028
FTHB_FLAG            -0.2695      0.017    -15.632      0.000      -0.303      -0.236
PUR_Cash_out          0.4305        nan        nan        nan         nan         nan
PUR_Refinance         0.5627        nan        nan        nan         nan         nan
PUR_Purchase          0.4658        nan        nan        nan         nan         nan
PRO_Condominium       0.2705        nan        nan        nan         nan         nan
PRO_Co_operative      0.1970        nan        nan        nan         nan         nan
PRO_Planned_Urban     0.2483        nan        nan        nan         nan         nan
PRO_Manufact_Home     0.4692        nan        nan        nan         nan         nan
PRO_Single_Family     0.2739        nan        nan        nan         nan         nan
NUM_UNIT             -0.0036      0.096     -0.038      0.970      -0.192       0.185
OCC_Principal         0.3265        nan        nan        nan         nan         nan
OCC_Second            0.4254        nan        nan        nan         nan         nan
OCC_Investor          0.7071        nan        nan        nan         nan         nan
MI_PCT               -0.1379      0.069     -1.993      0.046      -0.274      -0.002
MI_Borrower           0.1765      0.048      3.683      0.000       0.083       0.270
MI_Lender             0.3188      0.164      1.939      0.053      -0.003       0.641
ORIG_DTE_YR          -0.1416      0.411     -0.345      0.730      -0.947       0.663
ORIG_DTE_MONTH        0.0417      0.128      0.325      0.745      -0.209       0.293
FRST_DTE_YR          -2.2481      0.295     -7.609      0.000      -2.827      -1.669
FRST_DTE_MONTH       -1.0640      0.129     -8.245      0.000      -1.317      -0.811
=====================================================================================
"""

In [11]:
# WIP

# SVM

In [12]:
from sklearn import svm


In [14]:

clf = svm.SVC(kernel='linear') # Linear Kernel
clf.fit(X_train, Y_train)
y_pred = clf.predict(X_test)
print("Accuracy:", accuracy_score(Y_test, y_pred))



Accuracy: 0.9812510307294816


In [16]:
print(
    classification_report(
        y_pred=y_pred,
        y_true=Y_test
    )
)

              precision    recall  f1-score   support

         0.0       0.98      1.00      0.99    285599
         1.0       0.00      0.00      0.00      5457

    accuracy                           0.98    291056
   macro avg       0.49      0.50      0.50    291056
weighted avg       0.96      0.98      0.97    291056



/Users/melina/Documents/Uni/Fall 2024/MIE479/msf-capstone-crm/venv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/melina/Documents/Uni/Fall 2024/MIE479/msf-capstone-crm/venv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/melina/Documents/Uni/Fall 2024/MIE479/msf-capstone-crm/venv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_

In [13]:
weight_0 = len(Y_train)/(2*(len(Y_train)-sum(Y_train)))
weight_1 = len(Y_train)/(2* sum(Y_train))
weights = [ weight_1 if item > 0 else weight_0 for item in Y_train]

clf_b = svm.SVC(kernel='rbf') # Linear Kernel
clf_b.fit(X_train, Y_train, sample_weight=weights)
y_pred = clf_b.predict(X_test)
print("Accuracy:", accuracy_score(Y_test, y_pred))


In [ ]:
# SMOTE
smote_svm = svm.SVC(kernel='linear') # Linear Kernel
smote_svm.fit(X_train_smote, Y_train_smote)
y_pred = smote_svm.predict(X_test)
print("Accuracy:", accuracy_score(Y_test, y_pred))
print(
    classification_report(
        y_pred=y_pred,
        y_true=Y_test
    )
)

smote_svm_rbf = svm.SVC(kernel='rbf') # Linear Kernel
smote_svm_rbf.fit(X_train_smote, Y_train_smote)
y_pred = smote_svm_rbf.predict(X_test)
print("Accuracy:", accuracy_score(Y_test, y_pred))
print(
    classification_report(
        y_pred=y_pred,
        y_true=Y_test
    )
)

In [ ]:

smote_svm = svm.SVC(kernel='linear') # Linear Kernel
smote_svm.fit(pd.DataFrame(X_train_smote["CSCORE_B"]), Y_train_smote)
y_pred = clf.predict(pd.DataFrame(X_test["CSCORE_B"]))
print("Accuracy:", accuracy_score(Y_test, y_pred))
print(
    classification_report(
        y_pred=y_pred,
        y_true=Y_test
    )
)